In [11]:
import os
import numpy as np
from astropy.io import fits
import shutil

def check_and_delete_subfolders_with_holes(parent_folder, threshold=4.0):
    """
    Check subfolders in the parent folder, and delete the entire subfolder if any FITS file in it has holes exceeding the threshold.
    """
    for subfolder in os.listdir(parent_folder):
        subfolder_path = os.path.join(parent_folder, subfolder)

        # Ensure we're processing a directory
        if not os.path.isdir(subfolder_path):
            continue

        remove_subfolder = False

        # Check each FITS file in the subfolder
        for file in os.listdir(subfolder_path):
            if file.endswith('.fits'):
                file_path = os.path.join(subfolder_path, file)
                try:
                    with fits.open(file_path) as hdul:
                        data = hdul[0].data
                        if data is not None:
                            total_pixels = data.size
                            nan_pixels = np.sum(np.isnan(data))
                            nan_percentage = (nan_pixels / total_pixels) * 100
                            if nan_percentage > threshold:
                                remove_subfolder = True
                                break
                except Exception as e:
                    print(f"Error processing {file} in subfolder '{subfolder}': {e}")
                    remove_subfolder = True  # Assume deletion if there's an issue with any file
                    break

        # Remove the subfolder if any FITS file fails the threshold
        if remove_subfolder:
            print(f"Deleting subfolder '{subfolder_path}'")
            shutil.rmtree(subfolder_path)


your_parent_folder_path = "L3a"
check_and_delete_subfolders_with_holes(your_parent_folder_path)